# Project Planning

- Describe the project and goals.

- Task out how you will work through the pipeline in as much detail as you need to keep on track.

- Incluce a data dictionary.

- Clearly state your starting hypotheses (and add the testing of these to your task list).

## Goals

- Find drivers for customer churn.

- Construct a ML classification model that accurately predicts customer churn.

- Create modules that make your process repeateable.

- Document your process well enough to be presented or read like a report.



## Audience

- Your target audience for your notebook walkthrough is the Codeup Data Science team. This should guide your language and level of explanations in your walkthrough.

## Project Specifications

#### Why are our customers churning?

###### Some questions to think about include but are not limited to:

- Are there clear groupings where a customer is more likely to churn?

    - What if you consider contract type?
    - Is there a tenure that month-to-month customers are most likely to churn? 1-year contract customers? 2-year contract customers?
    - Do you have any thoughts on what could be going on? (Be sure to state these thoughts not as facts but as untested hypotheses. Unless you test them!). Plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers).

- Are there features that indicate a higher propensity to churn?

    - How influential are type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?

- Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point?

    - If so, what is that point and for which service(s)?

    - If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?


### Acquisition

- Acquire data from the customers table from the telco_churn database on the codeup data science database server.

- You will want to join some tables as part of your query.

- This data should end up in a pandas data frame.

- summarize data (.info(), .describe(), .value_counts(), ...)

- plot distributions of individual variables



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from acquire import get_telco_data

In [2]:
#grabbing telco data from SQL using function and storing it as a DataFrame
df = pd.DataFrame(get_telco_data())
df = df.set_index('customer_id')

In [3]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
customer_id,,,,,,,,,,,,,,,,,,,,,
0003-MKNFE,2,1,1,Male,0,No,No,9,Yes,Yes,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
0013-MHZWF,4,1,1,Female,0,No,Yes,9,Yes,No,...,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
0015-UOCOJ,1,1,1,Female,1,No,No,7,Yes,No,...,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
0023-HGHWL,1,1,1,Male,1,No,No,1,No,No phone service,...,No,No,No,Yes,25.1,25.1,Yes,Month-to-month,DSL,Electronic check
0032-PGELS,3,1,1,Female,0,Yes,Yes,1,No,No phone service,...,No,No,No,No,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0003-MKNFE to 9975-GPKZU
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   gender                    7043 non-null   object 
 4   senior_citizen            7043 non-null   int64  
 5   partner                   7043 non-null   object 
 6   dependents                7043 non-null   object 
 7   tenure                    7043 non-null   int64  
 8   phone_service             7043 non-null   object 
 9   multiple_lines            7043 non-null   object 
 10  online_security           7043 non-null   object 
 11  online_backup             7043 non-null   object 
 12  device_protection         7043 non-null   object 
 13  tech_support              7043 non-null   object 
 14

__All of the columns do not have any null values__

In [5]:
df.describe()

,payment_type_id,internet_service_type_id,contract_type_id,senior_citizen,tenure,monthly_charges
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,2.315633,1.872923,1.690473,0.162147,32.371149,64.761692
std,1.148907,0.737796,0.833755,0.368612,24.559481,30.090047
min,1.000000,1.000000,1.000000,0.000000,0.000000,18.250000
25%,1.000000,1.000000,1.000000,0.000000,9.000000,35.500000
50%,2.000000,2.000000,1.000000,0.000000,29.000000,70.350000
75%,3.000000,2.000000,2.000000,0.000000,55.000000,89.850000
max,4.000000,3.000000,3.000000,1.000000,72.000000,118.750000


__There does not seem to be any outliers__

In [6]:
df.payment_type_id.value_counts()

1    2365
2    1612
3    1544
4    1522
Name: payment_type_id, dtype: int64

- 1: electronic check
- 2: mailed check
- 3: bank transfer(automatic)
- 4: credit card(automatic)

In [7]:
df.internet_service_type_id.value_counts()

2    3096
1    2421
3    1526
Name: internet_service_type_id, dtype: int64

- 1: DSL
- 2: Fiber Optic
- 3: None

In [8]:
df.contract_type_id.value_counts()

1    3875
3    1695
2    1473
Name: contract_type_id, dtype: int64

- 1: Month-to-month
- 2: 1 year
- 3: 2 year

In [9]:
df.senior_citizen.value_counts()

0    5901
1    1142
Name: senior_citizen, dtype: int64

In [10]:
df.phone_service.value_counts()

Yes    6361
No      682
Name: phone_service, dtype: int64

- 0: Is not senior citizen
- 1: Is senior citizen

__Confirmed there are no null values in senior citizens, payment, internet and contract type id__

In [ ]:
# distribution of columns whose data type is 'int64'
num_cols = df.columns[[df[col].dtype == 'int64' for col in df.columns]]
for col in num_cols:
    plt.hist(df[col])
    plt.title(col)
    plt.show()

__There is a normal distribution across payment type, internet service and tenure__

__There are a lot more customers who are not senior citizens than who are__

__There are more month-to-month customers than the 1 and 2 year subscriptions combined__

### Data Prep

- Change device_protection, tech_support and papperless_billing to 0/1

- Create a new feature that represents tenure in years.

- Create single variables for or find other methods to merge variables representing the information from the following columns:

    - phone_service and multiple_lines
    - dependents and partner
    - streaming_tv & streaming_movies
    - online_security & online_backup
    
- Split your data into train/validate/test.

In [ ]:
def online_checker(row):
        if row == 'Yes':
            return 2
        elif row == 'No':
            return 1
        elif row == "No internet service":
            return 0

In [ ]:
df.device_protection = df.device_protection.apply(online_checker)

In [ ]:
df.tech_support = df.tech_support.apply(online_checker)

In [ ]:
df['tenure_by_year'] = df.tenure / 12

In [ ]:
def phone_checker(row):
        if row == 'Yes':
            return 2
        elif row == 'No':
            return 1
        elif row == "No phone service":
            return 0

In [ ]:
df['multiple_line_values'] = df.multiple_lines.apply(phone_checker)

In [11]:
def family_checker(row):
        if row == 'Yes Yes':
            return 2
        elif row == 'Yes No' or row == 'No Yes':
            return 1
        elif row == 'No No':
            return 0

In [ ]:
df['part_or_dep_values'] = df['partner'].str.cat(df['dependents'], sep =" ") 

In [ ]:
df.part_or_dep_values = df.part_or_dep_values.apply(family_checker)

In [ ]:
def stream_checker(row):
        if row == 'Yes Yes':
            return 3
        elif row == 'Yes No' or row == 'No Yes':
            return 2
        elif row == 'No No':
            return 1
        elif row == "No internet service No internet service":
            return 0

In [ ]:
df['streaming_tv_or_movie'] = df['streaming_tv'].str.cat(df['streaming_movies'], sep =" ") 

In [ ]:
df.streaming_tv_or_movie = df.streaming_tv_or_movie.apply(stream_checker)

In [ ]:
df['security_or_backup_values'] = df.online_security.str.cat(df.online_backup, sep=" ")

In [ ]:
df.security_or_backup_values = df.security_or_backup_values.apply(stream_checker)

In [ ]:
telco_dummies = pd.get_dummies(df[['gender', 'churn', 'paperless_billing']], drop_first=True)

In [ ]:
df = pd.concat([df, telco_dummies], axis=1)

In [ ]:
col_to_drop = ['gender', 'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'streaming_tv', 'streaming_movies', 'churn', 'contract_type', 'internet_service_type', 'payment_type']

In [ ]:
df = df.drop(columns = col_to_drop)

In [ ]:
df

In [ ]:
def telco_split(df):

    train_validate, test = train_test_split(df, test_size=.15, 
                                        random_state=123, 
                                        stratify=df.churn_Yes)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.churn_Yes)
    return train, validate, test

In [ ]:
train, validate, test = telco_split(df)

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train.head()